### Building a RAG  HEF Assistant with Fabric

In [1]:
%pip install openai==1.12.0 azure-kusto-data langchain tenacity langchain-openai pypdf
%pip install beautifulsoup4 langchain-community

StatementMeta(, 9bc2ccbb-73b6-4883-b6fa-6375be8dd600, 8, Finished, Available, Finished)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 998.9 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 4.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-openai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 5.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 3.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
%pip install openai --upgrade

StatementMeta(, 9bc2ccbb-73b6-4883-b6fa-6375be8dd600, 14, Finished, Available, Finished)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 599.1/599.1 kB 3.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 351.8/351.8 kB 24.4 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.12.0
    Uninstalling openai-1.12.0:
      Successfully uninstalled openai-1.12.0

[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.



In [3]:
from openai import AzureOpenAI
from IPython.display import display, HTML
import os
import textwrap
import json 
import requests
import pandas as pd
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

from notebookutils import mssparkutils
from azure.kusto.data import KustoClient, KustoConnectionStringBuilder
from azure.kusto.data.exceptions import KustoServiceError
from azure.kusto.data.helpers import dataframe_from_result_table

from langchain.text_splitter import CharacterTextSplitter,RecursiveCharacterTextSplitter
from langchain_openai import AzureOpenAIEmbeddings
from langchain.document_loaders import PyPDFLoader
from langchain_community.document_loaders import WebBaseLoader
from tenacity import retry, wait_random_exponential, stop_after_attempt
from bs4 import SoupStrainer
from bs4 import BeautifulSoup

StatementMeta(, 9bc2ccbb-73b6-4883-b6fa-6375be8dd600, 16, Finished, Available, Finished)

In [ ]:
OPENAI_GPT4_DEPLOYMENT_NAME="gpt-4o-kenya-hack"
OPENAI_DEPLOYMENT_ENDPOINT="Your-OpenAI-Endpoint" # Replace with your OpenAI endpoint
OPENAI_API_KEY="Your-OpenAI-API-KEY" # Replace with your OpenAI API key
OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME = "text-embedding-ada-002-kenya-hack"


KUSTO_URI = 'Your-Kusto-URI' # Replace with your kusto URI
KUSTO_DATABASE = "HEF_eventhouse"
KUSTO_TABLE = "hefEmbeddings"
accessToken = mssparkutils.credentials.getToken(KUSTO_URI)

StatementMeta(, 9bc2ccbb-73b6-4883-b6fa-6375be8dd600, 17, Finished, Available, Finished)

##### Creating an Azure OpenAI client and defining a function to calculate embeddings

In [5]:
client = AzureOpenAI(
        azure_endpoint=OPENAI_DEPLOYMENT_ENDPOINT,
        api_key=OPENAI_API_KEY,
        api_version="2023-09-01-preview"
    )

#we use the tenacity library to create delays and retries when calling openAI embeddings to avoid hitting throttling limits
@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(6))
def generate_embeddings(text): 
    # replace newlines, which can negatively affect performance.
    txt = text.replace("\n", " ")
    return client.embeddings.create(input = [txt], model=OPENAI_ADA_EMBEDDING_DEPLOYMENT_NAME).data[0].embedding



StatementMeta(, 9bc2ccbb-73b6-4883-b6fa-6375be8dd600, 18, Finished, Available, Finished)

##### Reading the pdf files, divide it into 1000 chars chunks

In [6]:
# splitting into 1000 char long chunks with 30 char overlap
# split ["\n\n", "\n", " ", ""]
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=30,
)

# List of PDF files (adjust filenames as per your lakehouse)
pdf_files = [
    {"name": "UF-FAQs.pdf", "path": "/lakehouse/default/Files/UF-FAQs.pdf"},
    {"name": "HEF-NFM-FAQs.pdf", "path": "/lakehouse/default/Files/HEF-NFM-FAQs.pdf"},  
    {"name": "Helb-FAQS.pdf", "path": "/lakehouse/default/Files/Helb-FAQS.pdf"}, 
    {"name": "University-Funding-FAQs2.pdf", "path": "/lakehouse/default/Files/University-Funding-FAQs2.pdf"},
]

# List of web URLs
web_urls = [
    "https://www.helb.co.ke/faqs/students-faqs/",
    "https://www.helb.co.ke/faqs/loanees-faqs/",
    "https://www.helb.co.ke/faqs/employers-faqs/",
    "https://www.helb.co.ke/faqs/institutions-faqs/",
    "https://www.hef.co.ke/",
    "https://www.hef.co.ke/faqs/",
    "https://kuccps.net/frequently-asked-questions",
    "https://www.universitiesfund.go.ke/blog/frequently-asked-questions/",
]

StatementMeta(, 9bc2ccbb-73b6-4883-b6fa-6375be8dd600, 19, Finished, Available, Finished)

In [7]:
# Load PDFs
all_pages = []
for pdf in pdf_files:
    try:
        loader = PyPDFLoader(pdf["path"])
        pages = loader.load_and_split(text_splitter=splitter)
        print(f"Loaded {len(pages)} chunks from {pdf['name']}")
        all_pages.extend(pages)
    except Exception as e:
        print(f"Failed to load {pdf['name']}: {e}")

StatementMeta(, 9bc2ccbb-73b6-4883-b6fa-6375be8dd600, 20, Finished, Available, Finished)

Loaded 7 chunks from UF-FAQs.pdf
Loaded 8 chunks from HEF-NFM-FAQs.pdf
Loaded 12 chunks from Helb-FAQS.pdf
Loaded 19 chunks from University-Funding-FAQs2.pdf


In [9]:
# Load web content
for url in web_urls:
    try:
        # Try WebBaseLoader with SSL verification disabled
        loader = WebBaseLoader(url, verify_ssl=False)
        pages = loader.load_and_split(text_splitter=splitter)
        print(f"Loaded {len(pages)} chunks from {url}")
        all_pages.extend(pages)
    except Exception as e:
        print(f"WebBaseLoader failed for {url}: {e}")
        # Fallback: Use requests directly
        try:
            response = requests.get(url, verify=False)  # Bypass SSL verification
            response.raise_for_status()  # Check for HTTP errors
            soup = BeautifulSoup(response.text, "html.parser")
            text = soup.get_text(separator=" ")  # Extract all text
            
            # Create a single Document object manually
            from langchain.docstore.document import Document
            doc = Document(page_content=text, metadata={"source": url})
            pages = splitter.split_documents([doc])
            print(f"Fallback loaded {len(pages)} chunks from {url}")
            all_pages.extend(pages)
        except Exception as fallback_e:
            print(f"Fallback failed for {url}: {fallback_e}")

# Total chunks
print("Total number of chunks: ", len(all_pages))

StatementMeta(, 9bc2ccbb-73b6-4883-b6fa-6375be8dd600, 22, Finished, Available, Finished)

WebBaseLoader failed for https://www.helb.co.ke/faqs/students-faqs/: HTTPSConnectionPool(host='www.helb.co.ke', port=443): Max retries exceeded with url: /faqs/students-faqs/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')))
Fallback loaded 14 chunks from https://www.helb.co.ke/faqs/students-faqs/
WebBaseLoader failed for https://www.helb.co.ke/faqs/loanees-faqs/: HTTPSConnectionPool(host='www.helb.co.ke', port=443): Max retries exceeded with url: /faqs/loanees-faqs/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1006)')))
Fallback loaded 8 chunks from https://www.helb.co.ke/faqs/loanees-faqs/
WebBaseLoader failed for https://www.helb.co.ke/faqs/employers-faqs/: HTTPSConnectionPool(host='www.helb.co.ke', port=443): Max retries exceeded with url: /faqs/employers-faq

##### Saving the text chunks to a pandas dataframe

In [10]:
# Save to DataFrame
import pandas as pd
df = pd.DataFrame(columns=['document_name', 'content', 'embedding'])
for page in all_pages:
    # Use source (URL or file path) as document name
    doc_name = page.metadata.get('source', 'Unknown PDF')
    df.loc[len(df.index)] = [doc_name, page.page_content, ""]
df.head()

StatementMeta(, 9bc2ccbb-73b6-4883-b6fa-6375be8dd600, 23, Finished, Available, Finished)

,document_name,content,embedding
0,/lakehouse/default/Files/UF-FAQs.pdf,1. What is a Higher Education Government Stude...,
1,/lakehouse/default/Files/UF-FAQs.pdf,• The scholarship application is currently ope...,
2,/lakehouse/default/Files/UF-FAQs.pdf,and are placed by KUCCPS in a public universit...,
3,/lakehouse/default/Files/UF-FAQs.pdf,6. How can I apply for a Government Student Sc...,
4,/lakehouse/default/Files/UF-FAQs.pdf,8. How long can I qualify for the Government S...,


##### Calculating embeddings

In [11]:
# Generate embeddings (assuming generate_embeddings is defined)
df["embedding"] = df.content.apply(lambda x: generate_embeddings(x))
print(df.head(2))

# Optional: Save DataFrame to a file or database for later use
# df.to_csv("/lakehouse/default/Files/combined_embeddings.csv", index=False)

StatementMeta(, 9bc2ccbb-73b6-4883-b6fa-6375be8dd600, 24, Finished, Available, Finished)

                          document_name  \
0  /lakehouse/default/Files/UF-FAQs.pdf   
1  /lakehouse/default/Files/UF-FAQs.pdf   

                                             content  \
0  1. What is a Higher Education Government Stude...   
1  • The scholarship application is currently ope...   

                                           embedding  
0  [0.024698741734027863, -0.0024227292742580175,...  
1  [0.01745203323662281, 0.004714916460216045, 0....  


##### Writing the data to MS Fabric Eventhouse

In [12]:
df_sp = spark.createDataFrame(df)

df_sp.write.\
format("com.microsoft.kusto.spark.synapse.datasource").\
option("kustoCluster",KUSTO_URI).\
option("kustoDatabase",KUSTO_DATABASE).\
option("kustoTable", KUSTO_TABLE).\
option("accessToken", accessToken ).\
mode("Append").save()

StatementMeta(, 9bc2ccbb-73b6-4883-b6fa-6375be8dd600, 25, Finished, Available, Finished)

### Vector search on Fabric Eventhouse

##### A function to calling GPT4 for a Natural Language answer

In [13]:
def call_openAI(text):
    response = client.chat.completions.create(
        model=OPENAI_GPT4_DEPLOYMENT_NAME,
        messages = text,
        temperature=0
    )

    return response.choices[0].message.content

StatementMeta(, 9bc2ccbb-73b6-4883-b6fa-6375be8dd600, 26, Finished, Available, Finished)

#####  A function  retrieving answers using embeddings with similarity search

In [14]:
def get_answer_from_eventhouse(question, nr_of_answers=1):
        searchedEmbedding = generate_embeddings(question)
        kusto_query = KUSTO_TABLE + " | extend similarity = series_cosine_similarity(dynamic("+str(searchedEmbedding)+"), embedding) | top " + str(nr_of_answers) + " by similarity desc "
        kustoDf  = spark.read\
        .format("com.microsoft.kusto.spark.synapse.datasource")\
        .option("kustoCluster",KUSTO_URI)\
        .option("kustoDatabase",KUSTO_DATABASE)\
        .option("accessToken", accessToken)\
        .option("kustoQuery", kusto_query).load()

        return kustoDf

StatementMeta(, 9bc2ccbb-73b6-4883-b6fa-6375be8dd600, 27, Finished, Available, Finished)

In [15]:
# Retrieves 2 answers from Eventhouse
nr_of_answers = 2
question = "Can IGCSE graduates apply to KUCCPS for placement to universities and colleges?"
answers_df = get_answer_from_eventhouse(question, nr_of_answers)

# Concatenates the answers
answer = ""
for row in answers_df.rdd.toLocalIterator():
    answer = answer + " " + row['content']

# Creates a prompt for GPT4 with the question and the 2 answers
prompt = 'Question: {}'.format(question) + '\n' + 'Information: {}'.format(answer)
# prepare prompt
messages = [{"role": "system", "content": "You are a HELPFUL assistant answering users questions. Answer the question using the provided information and do not add anything else."},
            {"role": "user", "content": prompt}]

result = call_openAI(messages)
display(result)


StatementMeta(, 9bc2ccbb-73b6-4883-b6fa-6375be8dd600, 28, Finished, Available, Finished)

'No, IGCSE graduates cannot apply to KUCCPS for placement to universities and colleges. KUCCPS placement is specifically for students who have completed the KCSE examination.'